In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import re
import csv
from genres import genres

In [2]:
def dataset_webcraw(url):
    response = requests.get(url)
    res = BeautifulSoup(response.text) # 預設html.parser，不打也相同
    latest_data = res.find_all('div', class_ = 'download-item')[-1].a['href']
    return latest_data
url = 'https://data.gov.tw/dataset/59820'
response = requests.get(url)
res = BeautifulSoup(response.text)
dataset_lst = res.find_all('div', class_ = 'download-item')
dataset_eachfile = {}
data_df = {}
for i in dataset_lst:
    title = i.text.split('\n')[-1].strip()
    downlaodLink = i.a['href']
    dataset_eachfile[title] = downlaodLink
    data_df[title] = pd.read_csv(downlaodLink)

In [34]:
k = data_df['109年電影片分級及相關資訊']
k[k['國別'].str.contains('美國')]
# k[k['語言'] == 'Unknown']

,年度,中文片名,原文片名,國別,語言
6,2020,黑水風暴,DARK WATERS,美國英語,Unknown
12,2020,變身特務,Spies in Disguise,美國英語,Unknown
13,2020,深海終劫站,UNDERWATER,美國英語,Unknown
14,2020,絕地戰警FOR LIFE,BAD BOYS FOR LIFE,美國英語,Unknown
15,2020,杜立德,DOLITTLE,美國英語,Unknown
...,...,...,...,...,...
611,2020,末路狂奔,DREAMLAND,美國英語,Unknown
613,2020,末世戰疫：鳴鳥檔案,SONGBIRD,美國英語,Unknown
616,2020,致命天際線,SHADOW IN THE CLOUD,美國英語,Unknown
618,2020,不可侵犯：溫斯坦性醜聞,UNTOUCHABLE,美國英語,Unknown


# Clean dataset

In [3]:
# Check missing values
# 最後兩年有欄數不同與缺失
for v in data_df.values():
    print(v.isnull().sum().sum(), len(v.columns), sep =',')

0,7
0,7
0,7
0,7
0,7
0,7
0,9
1,9


In [4]:
# 演出時間有缺 但應該不需要這欄
data_df['111年電影片分級及相關資訊'][data_df['111年電影片分級及相關資訊'].isnull().any(axis=1)]

,年度,分級證明字號,級別,中文片名,原文片名,國別,語言,出品公司,映演時間
699,111,第111010號,普,一直一直都很喜歡你,Love Can't Be Said,大陸,國語,九洲音像出版公司、超娛影業（佛山）有限公司、超娛影業（深圳）有限公司、廣東珠影影視製作有限公...,NaN


In [5]:
# 處理104~109 欄位未分割問題
for title,m  in data_df.items():
    try:
        m['年度'] = m['年度'] + 1911
        m['中文片名'] = m['中文片名/外文片名'].apply(lambda row : row.split('\n')[0])
        m['原文片名'] = m['中文片名/外文片名'].apply(lambda row : row.split('\n')[1] if len(row.split('\n')) > 1 else 'No Eng Name')
        m['國別'] = m['國別/語別'].apply(lambda row : row.split('\n')[0])
        m['語言'] = m['國別/語別'].apply(lambda row : row.split('\n')[1] if len(row.split('\n')) > 1 else 'Unknown')
        df = m[['年度', '中文片名', '原文片名', '國別', '語言']]
        data_df[title] = df.drop_duplicates()
    except KeyError:
        df = m[['年度', '中文片名', '原文片名', '國別', '語言']]
        data_df[title] = df.drop_duplicates()

# check
for v in data_df.values():
    print(v.isnull().sum().sum(), len(v.columns), sep =',')

0,5
0,5
0,5
0,5
0,5
0,5
0,5
0,5


In [6]:
# 清理中文片名()內含有2D、3D、imax(大小寫)的部分
# 刪除現場XXXXX劇院現場的資料
# 去重複
pattern = r'\([^()]*?(2D|3D|IMAX|.*版)[^()]*?\)'
for key, df in data_df.items():
    df['中文片名'] = df['中文片名'].apply(lambda x: re.sub(pattern, '', x, flags=re.IGNORECASE))
    df = df[~df['中文片名'].str.contains('劇院現場')].reset_index(drop=True)
    data_df[key] = df.drop_duplicates()

# 新增標籤
---
這個會爬很久很久，目前已爬完存出去了，應該不會再用到

In [28]:
resultdf = []
errors = []
for i,df in enumerate(data_df.values()):
    new_column = []
    for index, row in df.iterrows():
        try:
            new_column.append(genres(row['中文片名']))
        except Exception as e:
            print("Error occurred in row:", index)
            print("Error occurred in row:", row['中文片名'], row['原文片名'])
            print("Error message:", str(e))
            new_column.append(None)
            errors.append((index, row['年度'], row['中文片名'], row['原文片名']))

    df['類型'] = new_column
    resultdf.append(df)
    df.to_csv(f'output{i}.csv', index=False)
    if errors:
        with open(f'error{i}.csv', 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['index', '年度', '中文片名', '原文片名']) 
            writer.writerows(errors)

Error occurred in row: 551
Error occurred in row: 削瘦的靈魂 A Lean Soul
Error message: 'NoneType' object has no attribute 'select'
Error occurred in row: 620
Error occurred in row: 願未央 Unfulfilled Dreams
Error message: 'NoneType' object has no attribute 'select'
Error occurred in row: 258
Error occurred in row: 邪門 Contorted
Error message: 'NoneType' object has no attribute 'select'
